In [1]:
%load_ext autoreload
%autoreload 2


# Tutorial Goal

This tutorial aims to show some example of **trace analysis and visualization**
using a pre-defined set of analysis and plotting functions provided by the
**Filters** and **Trace** modules of LISA.

In [2]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')
# Enable logging at INFO level
logging.getLogger().setLevel(logging.DEBUG)

In [3]:
# Generate plots inline
%matplotlib inline

# Python modules required by this notebook
import json
import os

# Configuration

In [4]:
# Let's use an example trace
res_dir = './example_results'
tracefile = os.path.join(res_dir, 'trace.dat')
platformfile = os.path.join(res_dir, 'platform.json')
!tree {res_dir}

./example_results
├── cluster_freqs.png
├── ediff_stats.png
├── ediff_time.png
├── platform.json
├── task_util_20331_sh.png
├── task_util_20552_chrome.png
├── task_util_20615_chrome.png
├── task_util_20672_keygen.png
├── task_util_20678_df.png
├── task_util_20687_chrome.png
├── task_util_20705_chrome.png
├── task_util_20803_sh.png
├── task_util_20805_lsof.png
├── task_util_650_permission_brok.png
├── trace.dat
├── trace.raw.txt
└── trace.txt

0 directories, 17 files


In [5]:
# Trace events of interest
events_to_parse = [
    "sched_switch",
    "sched_wakeup",
    "sched_wakeup_new",
    "sched_contrib_scale_f",
    "sched_load_avg_cpu",
    "sched_load_avg_task",
    "sched_tune_config",
    "sched_tune_tasks_update",
    "sched_tune_boostgroup_update",
    "sched_tune_filter",
    "sched_boost_cpu",
    "sched_boost_task",
    "sched_energy_diff",
    "cpu_frequency",
    "cpu_capacity",
]

# Platform description
with open(platformfile, 'r') as fh:
    platform = json.load(fh)

logging.info("CPUs max capacities:")
logging.info("   big: %5d (cpus: %s)",
             platform['nrg_model']['big']['cpu']['cap_max'],
             platform['clusters']['big'])
logging.info("LITTLE: %5d (cpus: %s)",
             platform['nrg_model']['little']['cpu']['cap_max'],
             platform['clusters']['little'])

# Time range from the analysis
(t_min, t_max) = (0, None)

02:17:27  INFO    : CPUs max capacities:
02:17:27  INFO    :    big:  1024 (cpus: [2, 3])
02:17:27  INFO    : LITTLE:   591 (cpus: [0, 1])


# Trace parsing

In [6]:
# Load the LISA::Trace parsing module
from trace import Trace


# The LISA::Trace module is a wrapper of the TRAPpy FTrace module which
# allows to keep track of platform specific details to support the generation
# of
trace = Trace(platform, res_dir, events_to_parse, window=(t_min,t_max))

02:17:27  DEBUG   : Loading [sched] events from trace in [./example_results]...
02:17:27  DEBUG   : Parsing events: ['sched_switch', 'sched_wakeup', 'sched_wakeup_new', 'sched_contrib_scale_f', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_tune_config', 'sched_tune_tasks_update', 'sched_tune_boostgroup_update', 'sched_tune_filter', 'sched_boost_cpu', 'sched_boost_task', 'sched_energy_diff', 'cpu_frequency', 'cpu_capacity']
02:17:27  INFO    : Parsing FTrace format...
02:18:02  DEBUG   : Events found on trace:
02:18:02  DEBUG   :  - sched_load_avg_task
02:18:02  DEBUG   :  - cpu_frequency
02:18:02  DEBUG   :  - cpu_capacity
02:18:02  DEBUG   :  - sched_tune_boostgroup_update
02:18:02  DEBUG   :  - sched_load_avg_cpu
02:18:02  DEBUG   :  - sched_boost_cpu
02:18:02  DEBUG   :  - sched_wakeup_new
02:18:02  DEBUG   :  - sched_tune_config
02:18:02  DEBUG   :  - sched_boost_task
02:18:02  DEBUG   :  - sched_tune_tasks_update
02:18:02  DEBUG   :  - sched_tune_filter
02:18:02  DEBUG   :  

Maximum estimated system energy: 3102


02:18:03  INFO    : Platform clusters verified to be Frequency coherent
02:18:03  DEBUG   : Lookup dataset for tasks...
02:18:03  INFO    : Collected events spans a 35.314 [s] time interval
02:18:03  INFO    : Set plots time range to (0.000000, 35.313536)[s]
02:18:03  DEBUG   : Registering [<trace.Trace object at 0x7fd32cfacc90>] local data frames
02:18:03  DEBUG   :    functions_stats
02:18:03  DEBUG   :    trace_event
02:18:03  DEBUG   :       Analysis - Analysis: /home/brejac01/sources/lisa/libs/utils/analysis
02:18:03  DEBUG   :       Analysis - Syspath: ['/home/brejac01/sources/lisa/libs/utils/analysis', '', '/home/brejac01/sources/lisa/libs/bart', '/home/brejac01/sources/lisa/libs/trappy', '/home/brejac01/sources/lisa/libs/devlib', '/home/brejac01/sources/lisa/libs/wlgen', '/home/brejac01/sources/lisa/libs/utils', '/home/brejac01/sources/lisa/ipynb/tutorial', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old

Notice how some platform specific data are collected and reported by the
LISA::Trace module

In [7]:
# This is the standard TRAPpy::FTrace object, already configured for the
# analysis related to the events of interest
ftrace = trace.ftrace
logging.info("List of events identified in the trace:")
for event in ftrace.class_definitions.keys():
    logging.info("   %s", event)

02:18:03  INFO    : List of events identified in the trace:
02:18:03  INFO    :    sched_load_avg_task
02:18:03  INFO    :    cpu_frequency
02:18:03  INFO    :    cpu_capacity
02:18:03  INFO    :    cpu_idle
02:18:03  INFO    :    sched_tune_boostgroup_update
02:18:03  INFO    :    sched_load_avg_cpu
02:18:03  INFO    :    sched_boost_cpu
02:18:03  INFO    :    sched_wakeup_new
02:18:03  INFO    :    sched_tune_config
02:18:03  INFO    :    sched_boost_task
02:18:03  INFO    :    sched_tune_tasks_update
02:18:03  INFO    :    sched_tune_filter
02:18:03  INFO    :    sched_energy_diff
02:18:03  INFO    :    sched_switch
02:18:03  INFO    :    sched_contrib_scale_f
02:18:03  INFO    :    sched_wakeup


In [8]:
# Original TRAPpy::FTrace DataSet are still accessible by specifying the
# trace event name of interest
trace.data_frame.trace_event('sched_load_avg_task').head()

,__comm,__cpu,__pid,comm,cpu,load_avg,load_sum,period_contrib,pid,util_avg,util_sum,cluster,min_cluster_cap
Time,,,,,,,,,,,,,
0.000014,<...>,1,20278,trace-cmd,1,58,2811439,916,20278,33,1601714,LITTLE,591
0.000229,<...>,1,20278,trace-cmd,1,59,2820191,111,20278,33,1612418,LITTLE,591
0.000334,<idle>,0,0,sh,0,0,0,957,20277,0,0,LITTLE,591
0.000982,sh,0,20277,sh,0,5,285169,710,20277,3,164585,LITTLE,591
0.001178,sh,0,20277,sh,0,5,355825,903,20277,3,205364,LITTLE,591


In [21]:
import trappy

In [25]:
pid = trace.getTaskByName("keygen")

In [32]:
ua = Parser(trace.ftrace, filters={"pid": pid}).solve("sched_load_avg_task:util_avg")

In [35]:
ua

,20672
Time,
14.917356,980
14.917371,980
14.917622,964
14.917718,964
14.919364,977
14.919745,956
14.920960,961
14.920999,961
14.921023,961


In [29]:
len(trace.analysis.tasks.getTaskSignal("keygen", "sched_load_avg_task:util_avg"))

2836

# LISA Tasks Filtering Functions

In [ ]:
trace.setXTimeRange(t_min, t_max)

## Top BIG tasks

In [ ]:
# Get a list of tasks which are the most big in the trace
top_big_tasks = trace.data_frame.top_big_tasks(
    min_utilization=None, # Minimum utilization to be considered "big"
                          # default: LITTLE CPUs max capacity
    min_samples=100,      # Number of samples over the minimum utilization 
)

In [ ]:
# The collected information is available for further analysis
top_big_tasks

In [ ]:
# Plot utilization of "big" tasks decorated with platform specific capacity information
trace.analysis.tasks.plotBigTasks()

## Top WAKEUP tasks

In [ ]:
top_wakeup_tasks = trace.data_frame.top_wakeup_tasks(
    min_wakeups=100  # Minimum number of wakeup to be reported
)

In [ ]:
top_wakeup_tasks.head()

### Wakeup vs Forks

In [ ]:
trace.analysis.tasks.plotWakeupTasks(per_cluster=False)

### Wakeup per cluster

In [ ]:
trace.analysis.tasks.plotWakeupTasks(per_cluster=True)

## RT Tasks

In [ ]:
trace.data_frame.rt_tasks(min_prio=100)

# Predefined LISA analysis Functions

**Trace** class provides an **analysis** object that allows to perform several types of analysis on data contained in the trace. Currently available analysis types are:

| Analysis Object | Description                           |
|-----------------|---------------------------------------|
| `cpus`          | CPUs Analysis                         |
| `eas`           | EAS-specific functionalities Analysis |
| `functions`     | Functions Profiling Analysis          |
| `frequency`     | Frequency Analysis                    |
| `status`        | System Status Analysis                |
| `tasks`         | Tasks Analysis                        |

 
 Those are easily accessible via:
 
 ```python
 trace.analysis.<analysis_object>
 ```

In [ ]:
# Define time ranges for all the time based plots
trace.setXTimeRange(t_min, t_max)

In [ ]:
trace.analysis.tasks.plotTasks(top_big_tasks.index.tolist())

In [ ]:
# Cluster frequencies
trace.analysis.frequency.plotClusterFrequencies()

In [ ]:
# Plots SchedTune's Energy-Diff Space Filtering
trace.analysis.eas.plotEDiffTime()